<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 3 Interlude: Chisel Standard Library
**Prev: [Generators: Collections](3.2_collections.ipynb)**<br>
**Next: [Higher-Order Functions](3.3_higher-order_functions.ipynb)**

## Motivation
Chisel 旨在重用，因此提供一个标准库的接口（鼓励 RTL 的互操作性）和常用硬件模块的生成器是非常合理的。

## Setup

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test

---
# The Cheatsheet
[Chisel3 cheatsheet](https://github.com/freechipsproject/chisel-cheatsheet/releases/latest/download/chisel_cheatsheet.pdf) 包含了所有主要硬件构建 API 的摘要，包括我们将在下面介绍的一些标准库工具。

# Decoupled: A Standard Ready-Valid Interface
Chisel 提供的常用接口之一是 `DecoupledIO`，它提供了一个 ready-valid 接口用于传输数据。其思想是，当有数据要传输时，source 驱动 `bits` 信号和 `valid` 信号。当 sink 准备好接收数据时，驱动 `ready` 信号。当 `ready` 和 `valid` 在同一个周期内都被置位时，数据被认为已经传输。

这为数据传输提供了一个双向的流量控制机制，包括一个反压机制。

注意：`ready` 和 `valid` 不应组合连接，否则可能会导致无法综合的组合环路。`ready` 只应依赖于 sink 是否能够接收数据，而 `valid` 只应依赖于 source 是否有数据。只有在事务完成后（在下一个时钟周期），这些值才会更新。

（译者注：一般来说，不能将 ready 信号置高作为置高 valid 的条件，但反之是可以的。此外，如果涉及到仲裁，例如 master 要向多个 slave 发送数据，则需要使用多个 ready 信号，此时 valid 信号可以根据 ready 信号的状态变化，但这个时候 ready 信号不能根据 valid 信号的状态变化，否则会形成组合环路。）

任何 Chisel 数据都可以包装在 `DecoupledIO` 中（用作 `bits` 字段），如下所示：

```scala
val myChiselData = UInt(8.W)
// 或任何 Chisel 数据类型，例如 Bool(), SInt(...)，甚至自定义 Bundles
val myDecoupled = Decoupled(myChiselData)
```

上面的代码创建了一个新的 `DecoupledIO` Bundle，具有以下字段：
- `valid`: Output(Bool)
- `ready`: Input(Bool)
- `bits`: Output(UInt(8.W))
___

本节的结构将与之前的有所不同：我们不会给你编程练习，而是会给出一些代码示例和测试用例，这些测试用例会打印电路状态。尝试在运行测试之前预测会打印出什么。

## Queues

`Queue` 创建了一个具有 Decoupled 接口的 FIFO（先进先出）队列，允许反压。数据类型和元素数量都是可配置的。

>注意我们在第一次调用 `peek` 的值时如何调用 `litValue`。这会将返回的 Chisel 字面量转换为 `BigInt`。稍后的类似调用我们不会调用 `litValue`，你可以看到 `peek` 返回的值的额外信息，比如类型和宽度。

In [ ]:
test(new Module {
    // Example circuit using a Queue
    val io = IO(new Bundle {
      val in = Flipped(Decoupled(UInt(8.W)))
      val out = Decoupled(UInt(8.W))
    })
    val queue = Queue(io.in, 2)  // 2-element queue
    io.out <> queue
  }) { c =>
    c.io.out.ready.poke(false.B)
    c.io.in.valid.poke(true.B)  // Enqueue an element
    c.io.in.bits.poke(42.U)
    println(s"Starting:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Enqueue another element
    c.io.in.bits.poke(43.U)
    // What do you think io.out.valid and io.out.bits will be?
    println(s"After first enqueue:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    c.io.in.valid.poke(true.B)  // Read a element, attempt to enqueue
    c.io.in.bits.poke(44.U)
    c.io.out.ready.poke(true.B)
    // What do you think io.in.ready will be, and will this enqueue succeed, and what will be read?
    println(s"On first read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    c.io.in.valid.poke(false.B)  // Read elements out
    c.io.out.ready.poke(true.B)
    // What do you think will be read here?
    println(s"On second read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)

    // Will a third read produce anything?
    println(s"On third read:")
    println(s"\tio.in: ready=${c.io.in.ready.peek().litValue}")
    println(s"\tio.out: valid=${c.io.out.valid.peek().litValue}, bits=${c.io.out.bits.peek().litValue}")
    c.clock.step(1)
}

## Arbiters
Arbiters 根据优先级从 _n_ 个 `DecoupledIO` 源将数据路由到一个 `DecoupledIO` sink。
Chisel 中包含两种类型：
- `Arbiter`: index 更低的 producers 优先
- `RRArbiter`: 以轮询顺序运行

注意，Arbiter 路由是在组合逻辑中实现的。

下面的示例将演示优先级仲裁器的使用（你将在下一节中实现这一点）：

In [ ]:
test(new Module {
    // Example circuit using a priority arbiter
    val io = IO(new Bundle {
      val in = Flipped(Vec(2, Decoupled(UInt(8.W))))
      val out = Decoupled(UInt(8.W))
    })
    // Arbiter doesn't have a convenience constructor, so it's built like any Module
    val arbiter = Module(new Arbiter(UInt(8.W), 2))  // 2 to 1 Priority Arbiter
    arbiter.io.in <> io.in
    io.out <> arbiter.io.out
  }) { c =>
    c.io.in(0).valid.poke(false.B)
    c.io.in(1).valid.poke(false.B)
    c.io.out.ready.poke(false.B)
    println(s"Start:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(1).valid.poke(true.B)  // Valid input 1
    c.io.in(1).bits.poke(42.U)
    c.io.out.ready.poke(true.B)
    // What do you think the output will be?
    println(s"valid input 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(0).valid.poke(true.B)  // Valid inputs 0 and 1
    c.io.in(0).bits.poke(43.U)
    // What do you think the output will be? Which inputs will be ready?
    println(s"valid inputs 0 and 1:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
    c.io.in(1).valid.poke(false.B)  // Valid input 0
    // What do you think the output will be?
    println(s"valid input 0:")
    println(s"\tin(0).ready=${c.io.in(0).ready.peek().litValue}, in(1).ready=${c.io.in(1).ready.peek().litValue}")
    println(s"\tout.valid=${c.io.out.valid.peek().litValue}, out.bits=${c.io.out.bits.peek().litValue}")
}

# Misc Function Blocks
Chisel Utils 有一些执行无状态函数的辅助函数。

## Bitwise Utilities
### PopCount
PopCount 返回输入中高（1）位的数量，类型为 `UInt`。

### Reverse
Reverse 返回比特反转的输入。

In [ ]:
test(new Module {
    // Example circuit using PopCount
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := PopCount(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("00000000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("11111111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

}

In [ ]:
test(new Module {
    // Example circuit using Reverse
    val io = IO(new Bundle {
      val in = Input(UInt(8.W))
      val out = Output(UInt(8.W))
    })
    io.out := Reverse(io.in)
  }) { c =>
    // Integer.parseInt is used create an Integer from a binary specification
    c.io.in.poke(Integer.parseInt("01010101", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("00001111", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11110000", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(Integer.parseInt("11001010", 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

## OneHot encoding utilities
OneHot 是一种整数编码方式，每个值都有一根导线，并且只有一根导线为高。这允许高效创建一些功能，例如 muxes。然而，如果不满足单导线高的条件，行为可能是未定义的。

下面两个函数提供了二进制 (`UInt`) 和 OneHot 编码之间的转换，它们是彼此的反函数：
- UInt 到 OneHot：`UIntToOH`
- OneHot 到 UInt：`OHToUInt`


In [ ]:
test(new Module {
    // Example circuit using UIntToOH
    val io = IO(new Bundle {
      val in = Input(UInt(4.W))
      val out = Output(UInt(16.W))
    })
    io.out := UIntToOH(io.in)
  }) { c =>
    c.io.in.poke(0.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(1.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(8.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")

    c.io.in.poke(15.U)
    println(s"in=${c.io.in.peek().litValue}, out=0b${c.io.out.peek().litValue.toInt.toBinaryString}")
}

In [ ]:
test(new Module {
    // Example circuit using OHToUInt
    val io = IO(new Bundle {
      val in = Input(UInt(16.W))
      val out = Output(UInt(4.W))
    })
    io.out := OHToUInt(io.in)
}) { c =>
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("0000 0000 1000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    c.io.in.poke(Integer.parseInt("1000 0000 0000 0001".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Some invalid inputs:
    // None high
    c.io.in.poke(Integer.parseInt("0000 0000 0000 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")

    // Multiple high
    c.io.in.poke(Integer.parseInt("0001 0100 0010 0000".replace(" ", ""), 2).U)
    println(s"in=0b${c.io.in.peek().litValue.toInt.toBinaryString}, out=${c.io.out.peek().litValue}")
}


## Muxes
这些 muxes 接受一个带有选择信号的值列表，并输出与最低索引选择信号关联的值。

它们可以接受 (select: Bool, value: Data) 元组列表，或作为参数传递的对应选择和值列表。为简单起见，下面的示例仅演示第二种形式。

### Priority Mux
`PriorityMux` 输出与最低索引置位选择信号关联的值。

### OneHot Mux
`Mux1H` 在保证只有一个选择信号为高时提供高效实现。如果假设不成立，则行为未定义。


In [ ]:
test(new Module {
    // Example circuit using PriorityMux
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := PriorityMux(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select higher index only
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both - arbitration needed
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select lower index only
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

In [ ]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val in_sels = Input(Vec(2, Bool()))
      val in_bits = Input(Vec(2, UInt(8.W)))
      val out = Output(UInt(8.W))
    })
    io.out := Mux1H(io.in_sels, io.in_bits)
  }) { c =>
    c.io.in_bits(0).poke(10.U)
    c.io.in_bits(1).poke(20.U)

    // Select index 1
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select index 0
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select none (invalid)
    c.io.in_sels(0).poke(false.B)
    c.io.in_sels(1).poke(false.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")

    // Select both (invalid)
    c.io.in_sels(0).poke(true.B)
    c.io.in_sels(1).poke(true.B)
    println(s"in_sels=${c.io.in_sels(0).peek().litValue}, out=${c.io.out.peek().litValue}")
}

## Counter
`Counter` 是一个计数器，可以每个周期递增一次，直到某个指定的限制，此时它会溢出。注意，它**不是**一个 Module，并且它的值是可访问的。

In [ ]:
test(new Module {
    // Example circuit using Mux1H
    val io = IO(new Bundle {
      val count = Input(Bool())
      val out = Output(UInt(2.W))
    })
    val counter = Counter(3)  // 3-count Counter (outputs range [0...2])
    when(io.count) {
      counter.inc()
    }
    io.out := counter.value
  }) { c =>
    c.io.count.poke(true.B)
    println(s"start: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 1: counter value=${c.io.out.peek().litValue}")

    c.clock.step(1)
    println(s"step 2: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(false.B)
    c.clock.step(1)
    println(s"step without increment: counter value=${c.io.out.peek().litValue}")

    c.io.count.poke(true.B)
    c.clock.step(1)
    println(s"step again: counter value=${c.io.out.peek().litValue}")
}

---
# You're done!

[Return to the top.](#top)